In [1]:
#Reading appropriate libraries 

import tensorflow as tf
from tensorflow import keras

#Library for binarized neural network
import larq as lq
import pandas as pd
import numpy as np
import random as rand
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

2022-12-02 15:26:24.423270: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 15:26:25.118449: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/saumya/.conda/envs/tf/lib/
2022-12-02 15:26:25.118529: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/saumya/.conda/envs/tf/lib/
2022-12-02 15:26:25.118537: W tensorflow/compiler/tf2tensorrt/

In [2]:
#Reading banknote authentication dataset for this code

df = pd.read_csv('/home/saumya/Downloads/banknote_authentication.csv', sep = ';')

In [3]:
#Extracting the needed information
xdata = df[['wavelet_transformed_variance', 'wavelet_transformed_skewness', 'wavelet_transformed_curtosis','image_entropy']].to_numpy()
ydata = df[['counterfeit']].to_numpy()

In [4]:
#Here the error from the approximate processors are simulated in the dataset.
#Here we are simulating a random error of +,- 10% 
def add_error(xdata):
    error_mat = np.ones(np.shape(xdata))
    for i in range(error_mat.shape[0]):
        for j in range(error_mat.shape[1]):
            error_mat[i][j] = xdata[i][j]*0.1*rand.random()*(-1)**(rand.randint(0, 1))
    return(error_mat)

In [5]:
#new data with added error on it
#simulating error for 10 approximate process
for i in range(10):
    error_mat = add_error(xdata)
    xdata_error = xdata + error_mat

In [6]:
#Fitting the data 
scaler = StandardScaler()
xtrain = scaler.fit_transform(xdata)
xtrain_error = scaler.fit_transform(xdata_error)

In [7]:
#Splitting the data for trainning and testing
xtrain, xtest, ytrain, ytest = train_test_split(xdata, ydata, test_size = 0.2, train_size=0.8)
xtrain_error, xtest_error, ytrain_error, ytest_error = train_test_split(xdata_error, ydata, test_size = 0.2, train_size=0.8)

In [8]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")

#Defining model with no added error
model = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
#Input Layer
model.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
#Second Layer
model.add(lq.layers.QuantDense(8, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
#Third Layer
model.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
#Output layer
model.add(lq.layers.QuantDense(1,activation='sigmoid', use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))

2022-12-02 15:26:26.219460: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 15:26:26.224655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 15:26:26.224854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 15:26:26.225385: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [9]:
#Defining the model to train with error added data

model_error = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
#Input Layer
model_error.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))
#Second Layer
model_error.add(lq.layers.QuantDense(8, use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))
#Third Layer
model_error.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))
#Output layer
model_error.add(lq.layers.QuantDense(1,activation='sigmoid', use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))

In [10]:
eph_step = 2000

In [11]:
#Compiling the model with no added error

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(xtrain, ytrain, batch_size=64, epochs=eph_step)

test_loss, test_acc = model.evaluate(xtest, ytest)

Epoch 1/2000
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2022-12-02 15:26:28.762551: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1d9288c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-12-02 15:26:28.762586: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2022-12-02 15:26:28.766833: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-12-02 15:26:28.830152: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-02 15:26:28.863438: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


18/18 [==============================] - 3s 5ms/step - loss: 6.1948 - accuracy: 0.4667
Epoch 2/2000
18/18 [==============================] - 0s 5ms/step - loss: 6.3748 - accuracy: 0.4439
Epoch 3/2000
18/18 [==============================] - 0s 5ms/step - loss: 5.1928 - accuracy: 0.4485
Epoch 4/2000
18/18 [==============================] - 0s 4ms/step - loss: 4.5132 - accuracy: 0.5734
Epoch 5/2000
18/18 [==============================] - 0s 4ms/step - loss: 5.7216 - accuracy: 0.5169
Epoch 6/2000
18/18 [==============================] - 0s 4ms/step - loss: 6.0634 - accuracy: 0.5187
Epoch 7/2000
18/18 [==============================] - 0s 4ms/step - loss: 5.8035 - accuracy: 0.4968
Epoch 8/2000
18/18 [==============================] - 0s 4ms/step - loss: 6.5193 - accuracy: 0.4758
Epoch 9/2000
18/18 [==============================] - 0s 4ms/step - loss: 4.1771 - accuracy: 0.5506
Epoch 10/2000
18/18 [==============================] - 0s 4ms/step - loss: 6.0193 - accuracy: 0.4840
Epoch 11/200

18/18 [==============================] - 0s 3ms/step - loss: 2.1631 - accuracy: 0.7065
Epoch 83/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.1278 - accuracy: 0.7001
Epoch 84/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.0398 - accuracy: 0.7119
Epoch 85/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.1532 - accuracy: 0.7028
Epoch 86/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.2359 - accuracy: 0.7165
Epoch 87/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.2264 - accuracy: 0.6727
Epoch 88/2000
18/18 [==============================] - 0s 3ms/step - loss: 2.3506 - accuracy: 0.6910
Epoch 89/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.0041 - accuracy: 0.7183
Epoch 90/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.0959 - accuracy: 0.6800
Epoch 91/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.9998 - accuracy: 0.6727
Epoc

18/18 [==============================] - 0s 4ms/step - loss: 4.1311 - accuracy: 0.6308
Epoch 163/2000
18/18 [==============================] - 0s 4ms/step - loss: 4.4379 - accuracy: 0.6126
Epoch 164/2000
18/18 [==============================] - 0s 3ms/step - loss: 4.3276 - accuracy: 0.6253
Epoch 165/2000
18/18 [==============================] - 0s 3ms/step - loss: 4.7543 - accuracy: 0.6026
Epoch 166/2000
18/18 [==============================] - 0s 4ms/step - loss: 5.3508 - accuracy: 0.6080
Epoch 167/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.5784 - accuracy: 0.6846
Epoch 168/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.4064 - accuracy: 0.7156
Epoch 169/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.9102 - accuracy: 0.7220
Epoch 170/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.6215 - accuracy: 0.6582
Epoch 171/2000
18/18 [==============================] - 0s 3ms/step - loss: 3.2611 - accuracy: 0.

...

18/18 [==============================] - 0s 4ms/step - loss: 1.1765 - accuracy: 0.6755
Epoch 1675/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2805 - accuracy: 0.7539
Epoch 1676/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2290 - accuracy: 0.7356
Epoch 1677/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4421 - accuracy: 0.7293
Epoch 1678/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3488 - accuracy: 0.7612
Epoch 1679/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3449 - accuracy: 0.7885
Epoch 1680/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3912 - accuracy: 0.7457
Epoch 1681/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2589 - accuracy: 0.7521
Epoch 1682/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4705 - accuracy: 0.7812
Epoch 1683/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5877 - acc

18/18 [==============================] - 0s 5ms/step - loss: 1.4427 - accuracy: 0.8040
Epoch 1754/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5660 - accuracy: 0.7840
Epoch 1755/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.9004 - accuracy: 0.7885
Epoch 1756/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5679 - accuracy: 0.7849
Epoch 1757/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4592 - accuracy: 0.7922
Epoch 1758/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.4064 - accuracy: 0.7265
Epoch 1759/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5111 - accuracy: 0.8405
Epoch 1760/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.2862 - accuracy: 0.6946
Epoch 1761/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.4088 - accuracy: 0.7657
Epoch 1762/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4711 - acc

18/18 [==============================] - 0s 4ms/step - loss: 2.5908 - accuracy: 0.6974
Epoch 1833/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4113 - accuracy: 0.8159
Epoch 1834/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5369 - accuracy: 0.8268
Epoch 1835/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5350 - accuracy: 0.8423
Epoch 1836/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5148 - accuracy: 0.8222
Epoch 1837/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4808 - accuracy: 0.8368
Epoch 1838/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4968 - accuracy: 0.8432
Epoch 1839/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4261 - accuracy: 0.8259
Epoch 1840/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.1808 - accuracy: 0.7894
Epoch 1841/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2889 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.3480 - accuracy: 0.7721
Epoch 1912/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5663 - accuracy: 0.7612
Epoch 1913/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.1946 - accuracy: 0.7812
Epoch 1914/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3846 - accuracy: 0.7475
Epoch 1915/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4211 - accuracy: 0.7429
Epoch 1916/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3656 - accuracy: 0.7375
Epoch 1917/2000
18/18 [==============================] - 0s 3ms/step - loss: 1.3713 - accuracy: 0.7785
Epoch 1918/2000
18/18 [==============================] - 0s 3ms/step - loss: 1.3533 - accuracy: 0.7375
Epoch 1919/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3347 - accuracy: 0.7302
Epoch 1920/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2698 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.4174 - accuracy: 0.7083
Epoch 1991/2000
18/18 [==============================] - 0s 3ms/step - loss: 1.2403 - accuracy: 0.7265
Epoch 1992/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2738 - accuracy: 0.7521
Epoch 1993/2000
18/18 [==============================] - 0s 3ms/step - loss: 1.4020 - accuracy: 0.7566
Epoch 1994/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4547 - accuracy: 0.7293
Epoch 1995/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3143 - accuracy: 0.7320
Epoch 1996/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2835 - accuracy: 0.6928
Epoch 1997/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5081 - accuracy: 0.7867
Epoch 1998/2000
18/18 [==============================] - 0s 3ms/step - loss: 1.4821 - accuracy: 0.7183
Epoch 1999/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3147 - acc

In [12]:
#Compiling the model with added error

model_error.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_error.fit(xtrain_error, ytrain_error, batch_size=64, epochs=eph_step)

test_loss_error, test_acc_error = model_error.evaluate(xtest_error, ytest_error)

Epoch 1/2000
18/18 [==============================] - 2s 4ms/step - loss: 5.7976 - accuracy: 0.5287
Epoch 2/2000
18/18 [==============================] - 0s 4ms/step - loss: 6.3211 - accuracy: 0.5160
Epoch 3/2000
18/18 [==============================] - 0s 4ms/step - loss: 5.2346 - accuracy: 0.5406
Epoch 4/2000
18/18 [==============================] - 0s 4ms/step - loss: 5.1850 - accuracy: 0.5378
Epoch 5/2000
18/18 [==============================] - 0s 4ms/step - loss: 4.7369 - accuracy: 0.5697
Epoch 6/2000
18/18 [==============================] - 0s 4ms/step - loss: 5.1542 - accuracy: 0.5232
Epoch 7/2000
18/18 [==============================] - 0s 4ms/step - loss: 4.3521 - accuracy: 0.5187
Epoch 8/2000
18/18 [==============================] - 0s 4ms/step - loss: 4.9478 - accuracy: 0.5415
Epoch 9/2000
18/18 [==============================] - 0s 4ms/step - loss: 5.1650 - accuracy: 0.5953
Epoch 10/2000
18/18 [==============================] - 0s 4ms/step - loss: 5.0766 - accuracy: 0.5561

18/18 [==============================] - 0s 4ms/step - loss: 3.9347 - accuracy: 0.5342
Epoch 83/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.7718 - accuracy: 0.5433
Epoch 84/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.9084 - accuracy: 0.5488
Epoch 85/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.6925 - accuracy: 0.5451
Epoch 86/2000
18/18 [==============================] - 0s 4ms/step - loss: 4.0060 - accuracy: 0.5223
Epoch 87/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.7433 - accuracy: 0.5652
Epoch 88/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.4899 - accuracy: 0.5488
Epoch 89/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.7401 - accuracy: 0.5779
Epoch 90/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.7480 - accuracy: 0.5679
Epoch 91/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.6780 - accuracy: 0.5697
Epoc

18/18 [==============================] - 0s 4ms/step - loss: 3.2651 - accuracy: 0.6263
Epoch 163/2000
18/18 [==============================] - 0s 3ms/step - loss: 3.2918 - accuracy: 0.6244
Epoch 164/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.1652 - accuracy: 0.6208
Epoch 165/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.2411 - accuracy: 0.6281
Epoch 166/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.3032 - accuracy: 0.6381
Epoch 167/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.2392 - accuracy: 0.6217
Epoch 168/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.3335 - accuracy: 0.6098
Epoch 169/2000
18/18 [==============================] - 0s 3ms/step - loss: 3.2442 - accuracy: 0.6509
Epoch 170/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.1506 - accuracy: 0.6007
Epoch 171/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.2104 - accuracy: 0.

18/18 [==============================] - 0s 4ms/step - loss: 3.3506 - accuracy: 0.6509
Epoch 243/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.3354 - accuracy: 0.6399
Epoch 244/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.2207 - accuracy: 0.6664
Epoch 245/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.2328 - accuracy: 0.6162
Epoch 246/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.4009 - accuracy: 0.6518
Epoch 247/2000
18/18 [==============================] - 0s 3ms/step - loss: 3.2073 - accuracy: 0.6490
Epoch 248/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.4286 - accuracy: 0.6354
Epoch 249/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.3180 - accuracy: 0.6326
Epoch 250/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.5067 - accuracy: 0.6481
Epoch 251/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.3745 - accuracy: 0.

18/18 [==============================] - 0s 4ms/step - loss: 3.1822 - accuracy: 0.6354
Epoch 323/2000
18/18 [==============================] - 0s 3ms/step - loss: 3.3016 - accuracy: 0.6153
Epoch 324/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.2531 - accuracy: 0.6190
Epoch 325/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.3067 - accuracy: 0.6326
Epoch 326/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.4039 - accuracy: 0.6290
Epoch 327/2000
18/18 [==============================] - 0s 3ms/step - loss: 3.2526 - accuracy: 0.6490
Epoch 328/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.2758 - accuracy: 0.6463
Epoch 329/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.2563 - accuracy: 0.6490
Epoch 330/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.4076 - accuracy: 0.6235
Epoch 331/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.2774 - accuracy: 0.

...

18/18 [==============================] - 0s 4ms/step - loss: 1.7794 - accuracy: 0.7639
Epoch 1675/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.0536 - accuracy: 0.8314
Epoch 1676/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5403 - accuracy: 0.7922
Epoch 1677/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6223 - accuracy: 0.8004
Epoch 1678/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4033 - accuracy: 0.8213
Epoch 1679/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6214 - accuracy: 0.7803
Epoch 1680/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5194 - accuracy: 0.7976
Epoch 1681/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5893 - accuracy: 0.8177
Epoch 1682/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5206 - accuracy: 0.8122
Epoch 1683/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.8234 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.8896 - accuracy: 0.8040
Epoch 1754/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6630 - accuracy: 0.8040
Epoch 1755/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6501 - accuracy: 0.7985
Epoch 1756/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.5992 - accuracy: 0.7183
Epoch 1757/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.2235 - accuracy: 0.7858
Epoch 1758/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.3500 - accuracy: 0.6937
Epoch 1759/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9376 - accuracy: 0.8140
Epoch 1760/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2365 - accuracy: 0.7995
Epoch 1761/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.1841 - accuracy: 0.8295
Epoch 1762/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.1994 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.8717 - accuracy: 0.7603
Epoch 1833/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3009 - accuracy: 0.7976
Epoch 1834/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.8204 - accuracy: 0.8077
Epoch 1835/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.3507 - accuracy: 0.8432
Epoch 1836/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.3047 - accuracy: 0.8423
Epoch 1837/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9015 - accuracy: 0.6946
Epoch 1838/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9095 - accuracy: 0.7748
Epoch 1839/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7548 - accuracy: 0.7776
Epoch 1840/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7502 - accuracy: 0.7557
Epoch 1841/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.8087 - acc

18/18 [==============================] - 0s 4ms/step - loss: 2.2968 - accuracy: 0.8241
Epoch 1912/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.2673 - accuracy: 0.8332
Epoch 1913/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.1918 - accuracy: 0.8332
Epoch 1914/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.2288 - accuracy: 0.8323
Epoch 1915/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9960 - accuracy: 0.8314
Epoch 1916/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.1199 - accuracy: 0.8277
Epoch 1917/2000
18/18 [==============================] - 0s 3ms/step - loss: 1.7471 - accuracy: 0.8022
Epoch 1918/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7972 - accuracy: 0.7703
Epoch 1919/2000
18/18 [==============================] - 0s 3ms/step - loss: 2.2355 - accuracy: 0.8213
Epoch 1920/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.0028 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.5429 - accuracy: 0.7721
Epoch 1991/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5543 - accuracy: 0.7876
Epoch 1992/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6590 - accuracy: 0.7912
Epoch 1993/2000
18/18 [==============================] - 0s 3ms/step - loss: 1.4800 - accuracy: 0.7630
Epoch 1994/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7555 - accuracy: 0.7876
Epoch 1995/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6844 - accuracy: 0.7903
Epoch 1996/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5594 - accuracy: 0.7949
Epoch 1997/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2886 - accuracy: 0.7247
Epoch 1998/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9785 - accuracy: 0.7785
Epoch 1999/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.3101 - acc

In [13]:
test_loss_error_real, test_acc_error_real = model_error.evaluate(xtest, ytest)

9/9 [==============================] - 0s 2ms/step - loss: 1.8991 - accuracy: 0.8109


In [14]:
test_loss_real_error, test_acc_real_error = model.evaluate(xtest_error, ytest_error)

9/9 [==============================] - 0s 2ms/step - loss: 1.5976 - accuracy: 0.8218


In [15]:
lq.models.summary(model)

+sequential stats--------------------------------------------------------------------+
| Layer                  Input prec.  Outputs  # 1-bit  # 32-bit  Memory  1-bit MACs |
|                              (bit)               x 1       x 1    (kB)             |
+------------------------------------------------------------------------------------+
| quant_dense                      1  (-1, 4)       16         0    0.00          16 |
| batch_normalization              -  (-1, 4)        0         8    0.03           0 |
| quant_dense_1                    1  (-1, 8)       32         0    0.00          32 |
| batch_normalization_1            -  (-1, 8)        0        16    0.06           0 |
| quant_dense_2                    1  (-1, 4)       32         0    0.00          32 |
| batch_normalization_2            -  (-1, 4)        0         8    0.03           0 |
| quant_dense_3                    1  (-1, 1)        4         0    0.00           4 |
| batch_normalization_3            -  (-1, 

In [16]:
#The output is still accuarate with any added variation in data.
print(f"The accuracy with no added random error is: {test_acc * 100:.2f} %")

The accuracy with no added random error is: 77.82 %


In [17]:
print(f"The accuracy with added random testing data on random model is:  {test_acc_error * 100:.2f} %")

The accuracy with added random testing data on random model is:  86.18 %


In [18]:
print(f"The accuracy with added random testing data on real model is:  {test_acc_error_real * 100:.2f} %")

The accuracy with added random testing data on real model is:  81.09 %


In [19]:
print(f"The accuracy with added real testing data on random error model is:  {test_acc_real_error * 100:.2f} %")

The accuracy with added real testing data on random error model is:  82.18 %
